In [25]:
# importa lid pyodbc
import pyodbc

In [26]:
# classe de conexão de dados
class Connection():
    server = '.\SQLEXPRESS'
    database = 'empresa'

    def connect(self):
        return pyodbc.connect(
            Trusted_Connection='yes', 
            driver='{ODBC Driver 17 for SQL Server}', 
            server=self.server,
            database=self.database)

In [27]:
# pegar a conexão com o banco de dados
db = Connection()
c = db.connect()

In [29]:
class Categoria:
    def __init__(self, connection):
        self.connection = connection

    def mountQuery(self, template):
        dml = "update Categoria set "
        string = ""

        for t in range(len(template)):
            mano = ""
            if (t + 1) == len(template):
                mano = "=? "
            else:
                mano = "=?, "
            string = string + template[t] + mano
        
        dml = dml + string + "where id = ?"
        return dml

    def create(self):
        ddl = '''CREATE TABLE Categoria(
            id int primary key identity(1,1),
            -- identity (inicio, incremento): especifica o valor inicial e de incremento.
            nome varchar(32),
            descricao text
        )'''
        
        with self.connection:
            self.connection.execute(ddl)

    def read(self):
        ddl = '''select * from Categoria'''

        with self.connection:
            cursor = self.connection.cursor()
            linha = cursor.execute(ddl).fetchall()
            return linha
    
    def insert(self):
        dml = "insert into Categoria (nome, descricao) values (?,?)"
        nome = input("Insira o nome da categoria: ")
        descricao = input("Insirea o nome da descricao: ")

        parametros = tuple((nome, descricao))
        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(dml, parametros)

    def update(self):
        identifier = input("Insira o id a ser alterado ou digite n para cancelar: ")
        if identifier == "n":
            return

        nome = input("Insira o novo nome da categoria ou deixe em brano para manter a existente: ")
        descricao = input("Insira a nova descricao ou deixe em brano para manter a existente: ")
        
        par = list()
        template = list()

        if nome:
            par.insert(len(par),  nome)
            template.insert(len(template) ,'nome')
        if descricao:
            par.insert(len(par),  descricao)
            template.insert(len(template) ,'descricao')

        dml = self.mountQuery(template)
        print(dml)
        par.insert(len(par), identifier)
        parametros = tuple(par)
    
        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(dml, parametros)

    def delete(self):
        identifier = input("Digite o id do objeto a ser removido: ")
        dml = "delete from Categoria where id = ?"

        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(dml, identifier)

In [30]:
class Vendedor:
    def __init__(self, connection):
        self.connection = connection

    def mountQuery(self, template):
        dml = "update Vendedor set "
        string = ""

        for t in range(len(template)):
            mano = ""
            if (t + 1) == len(template):
                mano = "=? "
            else:
                mano = "=?, "
            string = string + template[t] + mano
        
        dml = dml + string + "where cpf=?"
        return dml

    def create(self):
        ddl = '''CREATE TABLE Vendedor(
            cpf varchar (14) primary key,
            nome varchar (128),
            email varchar (128),
            telefone varchar (20),
            endereco varchar (256)
        )'''

        with self.connection:
            self.connection.execute(ddl)

    def read(self):
        ddl = '''select * from Vendedor'''

        with self.connection:
            cursor = self.connection.cursor()
            linha = cursor.execute(ddl).fetchall()
            return linha

    def insert(self):
        dml = "insert into Vendedor (cpf, nome, email, telefone, endereco) values (?,?,?,?,?)"
        cpf = input("Insira o cpf do vendedor - ")
        nome = input("Insira o nome do vendedor - ")
        email = input("Insira o email do vendedor - ")
        telefone = input("Insira o telefone do vendedor - ")
        endereco = input("Insira o endereco do vendedor - ")

        parametros = tuple((cpf, nome, email, telefone, endereco))

        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(dml, parametros)
    
    def update(self):
        identifier = input("Insira o cpf do vendedor a ser alterado ou digite n para cancelar: ")
        if identifier == "n":
            return
  
        nome = input("Insira o novo nome ou deixe em branco para manter o mesmo: ")
        email = input("Insira o novo email ou deixe em branco para manter o mesmo:  ")
        telefone = input("Insira o telefone ou deixe em branco para manter o mesmo: ")
        endereco = input("Insira o novo endereco ou deixe em branco para manter o mesmo: ")

        par = list()
        template = list()

        if nome:
            par.insert(len(par),  nome)
            template.insert(len(template) ,'nome')
        if email:
            par.insert(len(par) ,email)
            template.insert(len(template),'email')
        if telefone:
            par.insert(len(par),telefone)
            template.insert(len(template),'telefone')
        if endereco:
            par.insert(len(par),endereco)
            template.insert(len(template),'endereco')

        dml = self.mountQuery(template)
        par.insert(len(par), identifier)
        parametros = tuple(par)

        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(dml, parametros)

    def delete(self):
        identifier = input("Digite o cpf do vendedor a ser removido: ")
        dml = "delete from Vendedor where cpf = ?"

        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(dml, identifier)


In [31]:
class Produto:
    def __init__(self, connection):
        self.connection = connection

    def mountQuery(self, template):
        dml = "update Produto set "
        string = ""

        for t in range(len(template)):
            mano = ""
            if (t + 1) == len(template):
                mano = "=? "
            else:
                mano = "=?, "
            string = string + template[t] + mano
        
        dml = dml + string + "where id=?"
        return dml
    
    def create(self):
        ddl = '''CREATE TABLE Produto (
                id int primary key identity (1, 1), 
                -- identity (inicio, incremento): especifica o
                -- valor inicial e de incremento dos valores 
                -- gerados para a primary key.s
                nome varchar (128),
                descricao text,
                marca varchar (32),
                id_categoria int foreign key references Categoria (id)
                )
              '''
        with self.connection:
            self.connection.execute(ddl)

    def insert(self):
        nome = input("Insira o nome - ")
        descricao = input("Insira a descricao do produto - ")
        marca = input("Insira a marca do produto - ")
        id_categoria = input("Insira o id da categoria - ")

        dml = "insert into Produto (nome, descricao, marca, id_categoria) values (?,?,?,?)"
        parametros = tuple((nome, descricao, marca, id_categoria))

        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(dml, parametros)

    def delete(self):
        identifier = input("Insira o id do produto a ser deletado: ")
        dml = "delete from Produto where id = ?"

        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(dml, identifier)

    def update(self):
        identifier = input("Insira o id do produto a ser atualizado: ")
        nome = input("Insira o nome - ")
        descricao = input("Insira a descricao do produto ou deixe em branco para manter o mesmo: ")
        marca = input("Insira a marca do produto ou deixe em branco para manter o mesmo: ")
        id_categoria = input("Insira o id da categoria ou deixe em branco para manter o mesmo: ")

        par = list()
        template = list()

        if nome:
            par.insert(len(par),  nome)
            template.insert(len(template), 'nome')
        if descricao:
            par.insert(len(par), descricao)
            template.insert(len(template), 'descricao')
        if marca:
            par.insert(len(par), marca)
            template.insert(len(template), 'marca')
        if id_categoria:
            par.insert(len(par), id_categoria)
            template.insert(len(template),'id_categoria')

        dml = self.mountQuery(template)
        par.insert(len(par), identifier)
        parametros = tuple(par)

        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(dml, parametros)

    def read(self):
        ddl = '''select * from Produto'''

        with self.connection:
            cursor = self.connection.cursor()
            linha = cursor.execute(ddl).fetchall()
            print(linha)
            return linha


In [32]:
class VendedorProduto:
    def __init__(self, connection):
        self.connection = connection
    
    def mountQuery(self, template):
        dml = "update VendedorProduto set "
        string = ""

        for t in range(len(template)):
            mano = ""
            if (t + 1) == len(template):
                mano = "=? "
            else:
                mano = "=?, "
            string = string + template[t] + mano
        
        dml = dml + string + "where id=?"
        return dml

    def create(self):
        ddl = '''CREATE TABLE VendedorProduto (
                id int primary key identity (1, 1), 
                -- identity (inicio, incremento): 
                -- especifica o valor inicial e de incremento dos 
                -- valores gerados para a primary key.
                cpf_vendedor varchar (14) foreign key references Vendedor (cpf),
                id_produto int foreign key references Produto (id),
                quantidade int,
                preco decimal);
              '''
        with self.connection:
            self.connection.execute(ddl)

    def read(self):
        ddl = '''select * from VendedorProduto'''
        with self.connection:
            cursor = self.connection.cursor()
            linha = cursor.execute(ddl).fetchall()
            print(linha)
            return linha
    
    def insert(self):
        cpf_vendedor = input("Insira o cpf do vendedor: ")
        id_produto = input("Insira o id do produto: ")
        quantidade = input("Insira a quantidade do produto: ")
        preco = input("Insira preco: ")

        dml = "insert into VendedorProduto (cpf_vendedor, id_produto, quantidade, preco) values (?,?,?,?)"
        parametros = tuple((cpf_vendedor, id_produto, quantidade, preco))

        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(dml, parametros)

    def delete(self):
        identifier = input("Insira o id da venda a ser removida: ")

        dml = "delete from VendedorProduto where id = ?"
        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(dml, identifier)

    def update(self):
        identifier = input("Insira o id da venda a ser atualizado: ")
        cpf_vendedor = input("Insira o cpf do vendedor ou deixe em branco para manter o mesmo: ")
        id_produto = input("Insira a id do produto ou deixe em branco para manter o mesmo: ")
        quantidade = input("Insira a quantidade do produto ou deixe em branco para manter o mesmo: ")
        preco = input("Insira o preco ou deixe em branco para manter o mesmo: ")

        par = list()
        template = list()

        if cpf_vendedor:
            par.insert(len(par),  cpf_vendedor)
            template.insert(len(template), 'cpf_vendedor')
        if id_produto:
            par.insert(len(par), id_produto)
            template.insert(len(template), 'id_produto')
        if quantidade:
            par.insert(len(par), quantidade)
            template.insert(len(template), 'quantidade')
        if preco:
            par.insert(len(par), preco)
            template.insert(len(template),'preco')

        dml = self.mountQuery(template)
        par.insert(len(par), identifier)
        parametros = tuple(par)

        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(dml, parametros)